# Tech Intro

## Main tables

In [827]:
# Real Estate's objects
real_estate_list: list = [
    'ag_object',
    'ag_parse_ad',
    'ag_lead_object',
    'ag_presentation_object',
    'ag_object_platform',
    'ag_posting_ad_realty',
    'native_realty_filial',
    'foreign_realty_filial',
    'spr_metro',
    'spr_metro_connection',
    'ag_object_parking']
# Lead's objects
lead_list: list = [
    'leads',
    'lead_utm',
    'client_wishes',
    'leads_email_order',
    'request_from_site',
    'lead_finance',
    'leads_source_spr',
    'leads_calls_connection',
    'leads_pipeline',
    'leads_not_implemented_reason',
    'leads_status',
    'ag_pipeline',
    'ag_pipeline_status',
    'leads_meta',
    'leads_meta_fields',
    'leads_meta_group',
    'leads_contacts_connect',
    'leads_contact',
    'leads_contact_meta',
    'leads_contact_meta_fields']
# Deal's objects
deal_list: list = [
    # Deal
    'a_deal',
    'a_deal_meta',
    'deal_passport',
    'a_deal_finance',
    'a_deal_check',
    'a_deal_test',
    # Contractor
    'a_contractor',
    'a_contractor_meta',
    'a_deal_contractor',
    'a_deal_contractor_meta',
    # Object
    'a_object',
    'a_object_meta',
    'a_deal_object']

## Main Variables

In [828]:
# Address and name of your notebook
venv_path = "/home/bigdata/a.umarov/jupyter_P24"
# Exploratory Data Analysis(EDA) table (eda_table)
eda_table = deal_list[11] # You can generate full-EDA cycle if you want through the all of tables
                                # Just iterate it !
# Exploratory Data Analysis(EDA) obj as PySpark DataFrame
eda_df = None
# Set the date of checking
sample_data_date = '20221027'

## Useful command and modules (can be transform into the 'importlib'- smart import logic)

In [829]:
import os
import pandas as pd
# To data analyze need the module
import pyspark.pandas as ps
os.environ["PYSPARK_PYTHON"] = f"{venv_path}/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = f"{venv_path}/bin/python3"

import findspark
findspark.init("/opt/dev/spark/spark")
 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext, DataFrameNaFunctions, \
                        Window, DataFrame, DataFrameStatFunctions, \
                        Column, Row, GroupedData, functions, types
from pyspark.sql.functions import isnan, when, count, col


conf = SparkConf().setAppName("get_info_from_" + eda_table + "_table")
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
spark = SparkSession(sc)

W1120 02:27:36.862079 10390 process.cpp:2877] Attempted to spawn already running process version@10.62.4.11:44708
I1120 02:27:36.862977 57195 sched.cpp:241] Version: 1.11.0
I1120 02:27:36.863234 10570 sched.cpp:345] New master detected at master@10.62.4.8:5050
I1120 02:27:36.863308 10570 sched.cpp:365] No credentials provided. Attempting to register without authentication
I1120 02:27:36.864657 10554 sched.cpp:753] Framework registered with 89161c75-8817-4ae4-ab24-cba26241d6f9-75627


## Read the spec-file from Hadoop and show some data sample

In [830]:
eda_df = spark.read.parquet("/prod/source/p24/" + eda_table + "/datekey=" + sample_data_date)
eda_df.show(10, False)

+--------+------+---------------------------------------------------------------------------------------------------------------+-------------+--------------------+
|id      |object|key                                                                                                            |value        |updated             |
+--------+------+---------------------------------------------------------------------------------------------------------------+-------------+--------------------+
|10000012|196747|Регион                                                                                                         |Краснодарский|2022-07-08T08:03:59Z|
|10000036|196748|Площадь по внутреннему обмеру с учетом лоджий, балконов и веранд                                               |             |2022-07-08T08:05:31Z|
|10000049|196748|Гарантийный срок на технологическое и инженерное оборудование, входящее в состав объекта долевого строительства|             |2022-07-08T08:05:31Z|
|1000005 |

## Show columns' properties

In [831]:
print('Data overview')
eda_df.printSchema()

Data overview
root
 |-- id: string (nullable = true)
 |-- object: string (nullable = true)
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)
 |-- updated: string (nullable = true)



## Total rows (count)

In [832]:
eda_df.count()

8109002

## Simple func to execute and show the result of the SQL-instruction

In [833]:
def execute(select_instruction: str = "select count(*) from eda_sql") -> object:
    """
    Method to execute and show the result of your SQL-instruction
    :default -> "select count(*) from eda_sql"
    :param :string select_instruction: SQL statement
    :return: result of the SQL-operation as a table
    """
    return spark.sql(select_instruction).show(truncate = False)

## Create the temp-table-view from the DF (called as "eda_sql")

In [834]:
eda_df.createOrReplaceTempView("eda_sql")

## Check the method and temp-table

In [835]:
execute("select count(*) from eda_sql")

+--------+
|count(1)|
+--------+
|8109002 |
+--------+



## Detect missing values and abnormal zeroes and print it

###### Check-rule Matrix


|  Name of the rule    |Available                   
|----------------------|-------------------------|
|NULL/NaN/NA           |+                        |
|'Zero'-value          |+                        |
|'Empty-string'        |+                        |

### 1st solution (PySpark & Pandas version)
#### Max perfomance but harder to read & understand

##### Filter and count after - all the values in the  our EDA-DF

In [836]:
# NULL/NaN/NA values condition
eda_df_null_values = eda_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_null_values

,id,object,key,value,updated
0,0,0,0,374,0


In [837]:
# Zero values condition
eda_df_zero_values = eda_df.select([count(when(col(c) == 0, c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_zero_values

,id,object,key,value,updated
0,0,0,0,9008,0


In [838]:
# Empty-string values condition
eda_df_empty_values = eda_df.select([count(when(col(c) == '', c)).alias(c) \
                                     for c in eda_df.columns]).toPandas()
eda_df_empty_values

,id,object,key,value,updated
0,0,0,0,5424367,0


In [839]:
# Unlock the limit to display all the columns
pd.options.display.max_rows = None
# To transpose a dataframe - use .T method
eda_df_pandas = pd.concat([eda_df_null_values, eda_df_zero_values, eda_df_empty_values], axis = 'index').T
# To rename columns to anything else after the transpose
eda_df_pandas.columns = (['NULL','Zero','Empty_string'])
eda_df_pandas

,NULL,Zero,Empty_string
id,0,0,0
object,0,0,0
key,0,0,0
value,374,9008,5424367
updated,0,0,0


In [840]:
def set_result_to_percent(df: object = None) -> object:
    """
    Method to execute and show the result of your PandasDataFrame
    :default -> None-object
    :param :Pandas DF
    :return: DF after the conversion (absolute -> percent)
    """
    if df is None:
        return "Please incl. at least one Pandas DF as a parameter"
    return round(df * 100 / eda_df.count(), 3)

In [841]:
set_result_to_percent(eda_df_pandas)

,NULL,Zero,Empty_string
id,0.000,0.000,0.000
object,0.000,0.000,0.000
key,0.000,0.000,0.000
value,0.005,0.111,66.893
updated,0.000,0.000,0.000


### SQL-instructions

In [842]:
#Find dupes or misunderstanding or useless static values 3 244 006 total
for col in eda_df.columns:
    execute("select count(distinct("+ col + ")) from eda_sql")

+------------------+
|count(DISTINCT id)|
+------------------+
|8109002           |
+------------------+



+----------------------+
|count(DISTINCT object)|
+----------------------+
|177438                |
+----------------------+



+-------------------+
|count(DISTINCT key)|
+-------------------+
|47                 |
+-------------------+



+---------------------+
|count(DISTINCT value)|
+---------------------+
|242419               |
+---------------------+



+-----------------------+
|count(DISTINCT updated)|
+-----------------------+
|221735                 |
+-----------------------+



In [843]:
def execute_template(column: str ='id') -> object:
    """
    Method to execute and show the result of your SQL-instruction
    :default -> "ID incl."
    :param :string select_instruction: column name
    :return: result of the SQL-operation as a table
    """
    print(execute("SELECT DISTINCT(" + column + ")\
                FROM eda_sql\
                WHERE 1 = 1 \
                ORDER BY " + column + " DESC"))
    return execute("SELECT DISTINCT(" + column + ")\
                    FROM eda_sql\
                    WHERE 1 = 1 \
                    ORDER BY " + column + " ASC")

In [844]:
#Find dupes or misunderstanding or useless static values 3 244 006 total
for col in eda_df.columns:
    execute_template(col)

+-------+
|id     |
+-------+
|9999999|
|9999998|
|9999997|
|9999996|
|9999995|
|9999994|
|9999993|
|9999992|
|9999991|
|9999990|
|999999 |
|9999989|
|9999988|
|9999987|
|9999986|
|9999985|
|9999984|
|9999983|
|9999982|
|9999981|
+-------+
only showing top 20 rows

None


+--------+
|id      |
+--------+
|10000   |
|100000  |
|1000000 |
|10000000|
|10000001|
|10000002|
|10000003|
|10000004|
|10000005|
|10000006|
|10000007|
|10000008|
|10000009|
|1000001 |
|10000010|
|10000011|
|10000012|
|10000013|
|10000014|
|10000015|
+--------+
only showing top 20 rows



+------+
|object|
+------+
|99999 |
|99998 |
|99997 |
|99996 |
|99995 |
|99994 |
|99993 |
|99992 |
|99991 |
|99990 |
|9999  |
|99989 |
|99988 |
|99987 |
|99986 |
|99985 |
|99984 |
|99983 |
|99982 |
|99981 |
+------+
only showing top 20 rows

None


+------+
|object|
+------+
|100   |
|1000  |
|10000 |
|100000|
|100001|
|100002|
|100003|
|100004|
|100005|
|100006|
|100007|
|100008|
|100009|
|10001 |
|100010|
|100011|
|100012|
|100013|
|100014|
|100015|
+------+
only showing top 20 rows



+-------------------------------------------------------------------------------------------------------+
|key                                                                                                    |
+-------------------------------------------------------------------------------------------------------+
|Этаж(и), на котором(ых) расположено помещение                                                          |
|Уровень отделки                                                                                        |
|Улица_название                                                                                         |
|Улица                                                                                                  |
|Строительный номер объекта долевого участия в многоквартирном доме                                     |
|Строение_название                                                                                      |
|Строение селектор значение                   

+---------------------------------------------------------------------------------------------------------------+
|key                                                                                                            |
+---------------------------------------------------------------------------------------------------------------+
|Адрес прочее                                                                                                   |
|Вид                                                                                                            |
|Вид использования земель                                                                                       |
|Гарантийный срок на объект долевого строительства                                                              |
|Гарантийный срок на технологическое и инженерное оборудование, входящее в состав объекта долевого строительства|
|Жилая площадь по внутреннему обмеру                                                    

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                                                                                                                                                                                                      |
+-------------------------------------------------------------------------------------------------------------------------------------------

+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                      |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
|null                                                                                                                                                       |
|                                                                                                                                                           |
|\t                                                                                                                                                         |
|\t\tЗдание (Нежилое здание, Жилое строение без прав

+--------------------+
|updated             |
+--------------------+
|2022-10-26T15:46:51Z|
|2022-10-26T15:46:50Z|
|2022-10-26T15:43:58Z|
|2022-10-26T15:20:52Z|
|2022-10-26T15:19:20Z|
|2022-10-26T15:19:19Z|
|2022-10-26T15:14:56Z|
|2022-10-26T15:14:55Z|
|2022-10-26T15:01:40Z|
|2022-10-26T14:57:42Z|
|2022-10-26T14:54:45Z|
|2022-10-26T14:53:00Z|
|2022-10-26T14:50:03Z|
|2022-10-26T14:38:15Z|
|2022-10-26T14:34:13Z|
|2022-10-26T14:34:02Z|
|2022-10-26T14:32:28Z|
|2022-10-26T14:29:17Z|
|2022-10-26T14:11:15Z|
|2022-10-26T14:03:38Z|
+--------------------+
only showing top 20 rows

None


+--------------------+
|updated             |
+--------------------+
|2017-01-28T08:52:52Z|
|2017-01-30T02:48:23Z|
|2017-01-30T06:09:13Z|
|2017-01-30T06:31:35Z|
|2017-01-30T08:29:25Z|
|2017-01-30T08:46:27Z|
|2017-01-31T02:34:27Z|
|2017-01-31T04:38:28Z|
|2017-02-01T07:42:02Z|
|2017-02-04T04:36:58Z|
|2017-02-06T13:21:21Z|
|2017-02-06T13:21:22Z|
|2017-02-07T03:15:31Z|
|2017-02-07T06:44:22Z|
|2017-02-08T05:55:03Z|
|2017-02-08T11:09:07Z|
|2017-02-08T11:10:39Z|
|2017-02-09T11:15:25Z|
|2017-02-13T07:27:32Z|
|2017-02-13T09:52:09Z|
+--------------------+
only showing top 20 rows



## Bonus

### 2nd solution (Cycle and variables)
#### Slower but more simple and has more readability than the previous method 

### Create the list of variables to re-use if needed

In [ ]:
useless_columns = []
null_values = []
zero_values = []
empty_string = []
col_name = []

In [ ]:
for col in eda_df.columns:
    null = round((eda_df.filter(eda_df[col].isNull()).count()/eda_df.count())*100, 3)
    zero = round(eda_df.filter(eda_df[col]==0).count()/eda_df.count()*100, 3)
    empty = round(eda_df.filter(eda_df[col]=='').count()/eda_df.count()*100, 3)
    if null in [0, 100] and zero in [0, 100] and empty in [0, 100]:
        useless_columns.append(col)
        continue
    null_values.append(null)
    zero_values.append(zero)
    empty_string.append(empty)
    col_name.append(col)

In [ ]:
zipped = list(zip(null_values, zero_values, empty_string))
eda_df_pandas2 = pd.DataFrame(zipped, index=col_name, columns=['NULL', 'Zero', 'Empty'])
eda_df_pandas2

### Remove useless columns

In [ ]:
eda_df = eda_df.drop(*useless_columns)

In [ ]:
pd.DataFrame(eda_df.dtypes, columns = ['Column Name','Data type'])

In [826]:
spark.stop()

I1120 02:27:28.831347 10390 sched.cpp:2174] Asked to stop the driver
I1120 02:27:28.831444 10550 sched.cpp:1207] Stopping framework 89161c75-8817-4ae4-ab24-cba26241d6f9-75626
